
# CS591 Final Project - Stock Market Analysis

## Scrape Stock Information

In [1]:
from pandas import DataFrame
from pandas import Series
from urllib import urlopen
from bs4 import BeautifulSoup
from sklearn.preprocessing import normalize as norm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, preprocessing
import statsmodels.formula.api as sm

base_url = "http://finance.yahoo.com"
end_url = "+Historical+Prices"
allStockSymbols = {}
df = DataFrame(columns=['Date','Open','High','Low','Close','Volume','Adjusted Close'])
numDaysInJan = 31
numDaysInFeb = 29

In [2]:
'''
    Makes a dictionary of all stock symbols to check the inputted stock versus
'''
stocklists = open('companylist.csv')
firstLine = True
for line in stocklists:
    if firstLine == True:
        firstLine = False
        pass
    allStockSymbols[line.split(',')[0]] = True
stocklists.close()
stocklists = open('companylist1.csv')
firstLine = True
for line in stocklists:
    if firstLine == True:
        firstLine = False
        pass
    allStockSymbols[line.split(',')[0]] = True
stocklists.close()
stocklists = open('companylist2.csv')
firstLine = True
for line in stocklists:
    if firstLine == True:
        firstLine = False
        pass
    allStockSymbols[line.split(',')[0]] = True
stocklists.close()
stocklists = open('nasdaqlisted.txt')
firstLine = True
for line in stocklists:
    if firstLine == True:
        firstLine = False
        pass
    allStockSymbols[line.split('|')[0]] = True
stocklists.close()
stocklists = open('otherlisted.txt')
firstLine = True
for line in stocklists:
    if firstLine == True:
        firstLine = False
        pass
    symbol = line.split('|')[0]
    if "$" in symbol:
        pass
    else:
        allStockSymbols[symbol] = True
stocklists.close()

In [3]:
'''
    Get the symbol of the stock we are gathering data on.
'''
stock_symbol = input("Please enter the stock symbol you are interested about in quotes.\n")
while stock_symbol not in allStockSymbols:
    print("ERROR: NOT VALID STOCK SYMBOL, TRY AGAIN")
    stock_symbol = input("Please enter the stock symbol you are interested about in quotes.\n")

Please enter the stock symbol you are interested about in quotes.
"AAPL"


In [4]:
def extractInformation(htmlSource):
    soup = BeautifulSoup(htmlSource, "lxml")
    table = soup.find(class_="yfnc_datamodoutline1").find("table")
    tableContents = table.findAll("tr")
    for i in range(1, len(tableContents)-1):
        rowContents = tableContents[i].contents
        rowData = []
        for rowContent in rowContents:
            rowData.append(rowContent.contents[0])
        if len(rowData) < 7:
            pass
        else:
            df.loc[len(df)] = rowData
    return soup.find(rel="next")

In [5]:
nextPage = {'href' : "/q/hp?s=" + stock_symbol+end_url}
first = True
while nextPage is not None:
    url = base_url + nextPage['href']
    htmlSource = urlopen(url)
    nextPage = extractInformation(htmlSource)

## Predicting Stock's Quarter based on previous years patterns

In [6]:
'''
    Separate the data for the first 3 months by years
'''
janData = []
febData = []
marchData = []
iterrows = df.iterrows()
for i, row in iterrows:
    if 'Jan' in row['Date']:
        janData.append(row)
    if 'Feb' in row['Date']:
        febData.append(row)
    if 'Mar' in row['Date']:
        marchData.append(row)

In [7]:
'''
    Separate all the data by months
'''
i = 0
while "2015" not in janData[i]['Date'] :
    i = i + 1
jan2016 = janData[0:i]
olderJan = []
year = 2014
oldI = i
while i < len(janData):
    if str(year) in janData[i]['Date']:
        olderJan.append(janData[oldI:i])
        oldI = i
        year = year-1
    i = i + 1
olderJan.append(janData[oldI:i])

i = 0
while "2015" not in febData[i]['Date'] :
    i = i + 1
feb2016 = febData[0:i]
olderFeb = []
year = 2014
oldI = i
while i < len(febData):
    if str(year) in febData[i]['Date']:
        olderFeb.append(febData[oldI:i])
        oldI = i
        year = year-1
    i = i + 1
olderFeb.append(febData[oldI:i])
    
i = 0
while "2015" not in marchData[i]['Date'] :
    i = i + 1
march2016 = marchData[0:i]
olderMarch = []
year = 2014
oldI = i
while i < len(marchData):
    if str(year) in marchData[i]['Date']:
        olderMarch.append(marchData[oldI:i])
        oldI = i
        year = year-1
    i = i + 1
olderMarch.append(marchData[oldI:i])

'''
    Even out the number of months in each list
        Needed in case a company started in february or march and the data would be uneven.
'''
if len(olderMarch) > len(olderFeb):
    del olderMarch[-1]
elif len(olderFeb) > len(olderJan):
    del olderFeb[-1]
    del olderMarch[-1]

In [8]:
'''
    Normalize the data for each month and put it in order from first day of the month open, high, low, close 
    until the last day of the month
'''
def formatDataList(dataList):
    initList = []
    outputList = []
    for month in dataList:
        monthInfo = []
        for entry in month:
            monthInfo.append(float(entry['Close']))
            monthInfo.append(float(entry['Low']))
            monthInfo.append(float(entry['High']))
            monthInfo.append(float(entry['Open']))
        initList.append(norm(np.fliplr(np.asarray(monthInfo).reshape(1,-1)))[0])
    return initList[::-1]

def getMonthDates(dataList):
    initList = []
    outputList = []
    for month in dataList:
        monthDates = []
        for entry in month:
            monthDates.append(int(entry['Date'].split()[1][0:-1])) # grab only the number portion of the date
        initList.append(np.asarray(monthDates)[::-1])
    return initList[::-1]

janMonthInfo = formatDataList(olderJan)
janDates = getMonthDates(olderJan)
febMonthInfo = formatDataList(olderFeb)
febDates = getMonthDates(olderFeb)
marMonthInfo = formatDataList(olderMarch)
marDates = getMonthDates(olderMarch)

In [9]:
'''
    Ideas and Plan:
        Linear regression on open prices for each month
            take all the regressions for a month and combine them into one
            3 regressions at the end, use to predict behavior of the stock in 3 intervals
        
        calculate a variance/fluctuation/error for each month
            High-low = range
            range/open price gives us a fluctuation percentage for a day
            average fluction = summation of all fluctuation percentages divided by the length of the month list
            average across all years
        
        find highest and lowest value for the month
            compare to open price to get the lowest and highest deviation percentage
            average across all years to get average deviation from start of month
'''

'\n    Ideas and Plan:\n        Linear regression on open prices for each month\n            take all the regressions for a month and combine them into one\n            3 regressions at the end, use to predict behavior of the stock in 3 intervals\n        \n        calculate a variance/fluctuation/error for each month\n            High-low = range\n            range/open price gives us a fluctuation percentage for a day\n            average fluction = summation of all fluctuation percentages divided by the length of the month list\n            average across all years\n        \n        find highest and lowest value for the month\n            compare to open price to get the lowest and highest deviation percentage\n            average across all years to get average deviation from start of month\n'

In [10]:
'''
    Information parsing for each month
'''

# January parsing
janOpenPrices = []
janMonthFluctuations = []
janDeviationPercentage = []
for month in janMonthInfo:
    i = 0
    highest = 0.0
    lowest = 1.0
    fluctuation = 0.0
    opens = []
    while i < len(month):
        opens.append(month[i])
        if highest < month[i+1]:
            highest = month[i+1]
        if lowest > month[i+2]:
            lowest = month[i+2]
        fluctuation = fluctuation + (month[i+1] - month[i+2])/month[i] # (high - low) / open
        i = i + 4
    janOpenPrices.append(opens)
    janMonthFluctuations.append(fluctuation/len(month))
    janDeviationPercentage.append((highest/month[0], lowest/month[0]))

janFluctuation = np.mean(janMonthFluctuations)
h = 0.0
l = 0.0
for high, low in janDeviationPercentage:
    h = h + high
    l = l + low
janDeviation = (h/len(janDeviationPercentage), l/len(janDeviationPercentage))

#Information parsing for February
febOpenPrices = []
febMonthFluctuations = []
febDeviationPercentage = []
for month in febMonthInfo:
    i = 0
    highest = 0.0
    lowest = 1.0
    fluctuation = 0.0
    opens = []
    while i < len(month):
        opens.append(month[i])
        if highest < month[i+1]:
            highest = month[i+1]
        if lowest > month[i+2]:
            lowest = month[i+2]
        fluctuation = fluctuation + (month[i+1] - month[i+2])/month[i] # (high - low) / open
        i = i + 4
    febOpenPrices.append(opens)
    febMonthFluctuations.append(fluctuation/len(month))
    febDeviationPercentage.append((highest/month[0], lowest/month[0]))

febFluctuation = np.mean(febMonthFluctuations)
h = 0.0
l = 0.0
for high, low in febDeviationPercentage:
    h = h + high
    l = l + low
febDeviation = (h/len(febDeviationPercentage), l/len(febDeviationPercentage))

#Information parsing for March
marOpenPrices = []
marMonthFluctuations = []
marDeviationPercentage = []
for month in marMonthInfo:
    i = 0
    highest = 0.0
    lowest = 1.0
    fluctuation = 0.0
    opens = []
    while i < len(month):
        opens.append(month[i])
        if highest < month[i+1]:
            highest = month[i+1]
        if lowest > month[i+2]:
            lowest = month[i+2]
        fluctuation = fluctuation + (month[i+1] - month[i+2])/month[i] # (high - low) / open
        i = i + 4
    marOpenPrices.append(opens)
    marMonthFluctuations.append(fluctuation/len(month))
    marDeviationPercentage.append((highest/month[0], lowest/month[0]))

marFluctuation = np.mean(marMonthFluctuations)
h = 0.0
l = 0.0
for high, low in marDeviationPercentage:
    h = h + high
    l = l + low
marDeviation = (h/len(marDeviationPercentage), l/len(marDeviationPercentage))

### Fluctuation represents an average error percentage
### Deviation represents the average highest increase and lowest decrease from the open price of a month

In [125]:
print "January Fluctuation =", janFluctuation
print "January Highest Deviation =", janDeviation[0]
print "January Lowest Deviation =", janDeviation[1]
print "\nFebruary Fluctuation =", febFluctuation
print "February Highest Deviation =", febDeviation[0]
print "February Lowest Deviation =", febDeviation[1]
print "\nMarch Fluctuation =", marFluctuation
print "March Highest Deviation =", marDeviation[0]
print "March Lowest Deviation =", marDeviation[1]

January Fluctuation = 0.0095749908656
January Highest Deviation = 1.15137100573
January Lowest Deviation = 0.916769922996

February Fluctuation = 0.00817186885369
February Highest Deviation = 1.09771570949
February Lowest Deviation = 0.921968981055

March Fluctuation = 0.00804821230373
March Highest Deviation = 1.1017793796
March Lowest Deviation = 0.920379759324


In [11]:
'''
    Calculates slopes for each month based as the average of the regressions of past years
'''
janSlope = 0
for i in range(len(janOpenPrices)):
    m, b = np.polyfit(janDates[i], janOpenPrices[i], 1)
    janSlope = janSlope + m
janSlope = janSlope/len(janOpenPrices)

febSlope = 0
for i in range(len(febOpenPrices)):
    m, b = np.polyfit(febDates[i], febOpenPrices[i], 1)
    febSlope = febSlope + m
febSlope = febSlope/len(febOpenPrices)

marSlope = 0
for i in range(len(marOpenPrices)):
    m, b = np.polyfit(marDates[i], marOpenPrices[i], 1)
    marSlope = marSlope + m
marSlope = marSlope/len(marOpenPrices)

In [12]:
'''
    Makes lists of the open prices for the first 3 months of 2016
'''
jan2016Opens = []
for day in jan2016:
    jan2016Opens.append(float(day['Open']))
jan2016Opens = jan2016Opens[::-1]

feb2016Opens = []
for day in feb2016:
    feb2016Opens.append(float(day['Open']))
feb2016Opens = feb2016Opens[::-1]

mar2016Opens = []
for day in march2016:
    mar2016Opens.append(float(day['Open']))
mar2016Opens = mar2016Opens[::-1]

In [13]:
'''
    Gets all the dates the stock market was open for the first 3 months of 2016
'''
jan2016Dates = []
for day in jan2016:
    jan2016Dates.append(int(day['Date'].split()[1][0:-1]))
jan2016Dates = jan2016Dates[::-1]

feb2016Dates = []
for day in feb2016:
    feb2016Dates.append(int(day['Date'].split()[1][0:-1]) + numDaysInJan)
feb2016Dates = feb2016Dates[::-1]

mar2016Dates = []
for day in march2016:
    mar2016Dates.append(int(day['Date'].split()[1][0:-1]) + numDaysInJan + numDaysInFeb)
mar2016Dates = mar2016Dates[::-1]

In [14]:
'''
    Connect the lines to each other
'''
jan2016Dates.append(feb2016Dates[0])
jan2016Opens.append(feb2016Opens[0])

feb2016Dates.append(mar2016Dates[0])
feb2016Opens.append(mar2016Opens[0])

In [20]:
'''
    Normalize all the values for 2016
'''
allOpens = []
for entry in jan2016Opens:
    allOpens.append(entry)
    
for entry in feb2016Opens:
    allOpens.append(entry)
    
for entry in mar2016Opens:
    allOpens.append(entry)
    
allOpens= norm(np.asarray(allOpens).reshape(1,-1))[0]

jan2016Opens = allOpens[0:len(jan2016Opens)]
feb2016Opens = allOpens[len(jan2016Opens): len(jan2016Opens) + len(feb2016Opens)]
mar2016Opens = allOpens[len(jan2016Opens) + len(feb2016Opens):]

In [97]:
'''
    Plots the actual value of the first quarter of the 2016 market.
'''
plt.plot(jan2016Dates, jan2016Opens, 'red')
plt.axvline(x=jan2016Dates[-1], ls='dashed', color='black')
plt.text(16, .13625, 'January', fontsize=15)
plt.plot(feb2016Dates, feb2016Opens, 'red')
plt.axvline(x=feb2016Dates[-1], ls='dashed', color='black')
plt.text(44, .13625, 'February', fontsize=15)
plt.plot(mar2016Dates, mar2016Opens, 'red')
plt.text(74, .13625, 'March', fontsize=15)
plt.ylabel('Stock Value')
plt.xlabel('Days passed')
plt.title('Actual Stock values in 2016')
plt.axis([jan2016Dates[0], mar2016Dates[-1], 0.116, 0.140])
plt.show()

In [33]:
'''
    Makes the predicted values based on the pattern matching created by the regressions.
'''
predictedJan = []
for i in range(len(jan2016Dates)):
    if i == 0:
        predictedJan.append(jan2016Opens[0])
    else:
        predictedJan.append(jan2016Opens[0] + jan2016Dates[i-1]*janSlope)

predictedFeb = []
for i in range(len(feb2016Dates)):
    if i == 0:
        predictedFeb.append(predictedJan[-1])
    else:
        predictedFeb.append(predictedJan[-1] + (feb2016Dates[i-1]-numDaysInJan)*febSlope)
    
predictedMar = []
for i in range(len(mar2016Dates)):
    if i == 0:
        predictedMar.append(predictedFeb[-1])
    else:
        predictedMar.append(predictedFeb[-1] + (mar2016Dates[i-1]-numDaysInJan-numDaysInFeb)*marSlope)

In [98]:
plt.plot(jan2016Dates, predictedJan, 'red')
plt.axvline(x=jan2016Dates[-1], ls='dashed', color='black')
plt.text(16, .13625, 'January', fontsize=15)
plt.plot(feb2016Dates, predictedFeb, 'red')
plt.axvline(x=feb2016Dates[-1], ls='dashed', color='black')
plt.text(44, .13625, 'February', fontsize=15)
plt.plot(mar2016Dates, predictedMar, 'red')
plt.text(74, .13625, 'March', fontsize=15)
plt.ylabel('Predicted Stock Value')
plt.xlabel('Days passed')
plt.title('Predicted Stock values in 2016')
plt.axis([jan2016Dates[0], mar2016Dates[-1], 0.128, 0.140])
plt.show()

##    Find correlation of volume to stock price

In [115]:
df['Close'] = df['Close'].apply(float)
df['Open'] = df['Open'].apply(float)
df['diff'] = df[['Close']].sub(df['Open'], axis=0)
df['newVolume'] = df['Volume'].str.replace(',', '')

In [119]:
# Create x, where x the 'Volume' column's values as floats
x = df['newVolume'].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x.reshape(-1,1))

# Run the normalizer on the dataframe
df['df_normalized'] = DataFrame(x_scaled)

In [120]:
df.plot(style='o',x='df_normalized', y='diff', legend=False)

# fit with np.polyfit
x=df['df_normalized']
y=df['diff']
m, b = np.polyfit(x, y, 1) # x ~= -1.3301 in y = m*x + b
plt.plot(x, m*x + b, color='red')
plt.axis((0, .4, -10, 10))
plt.ylabel('Change in the Stocks Value')
plt.xlabel('Volume')
plt.title('Volume Correlation')
plt.show()

0       0.021581
1       0.014932
2       0.017910
3       0.016767
4       0.015318
5       0.017272
6       0.032657
7       0.024886
8       0.013523
9       0.017487
10      0.014318
11      0.015127
12      0.012540
13      0.016511
14      0.013906
15      0.014147
16      0.019989
17      0.013678
18      0.013711
19      0.024208
20      0.016453
21      0.010270
22      0.013600
23      0.013585
24      0.017240
25      0.018829
26      0.023260
27      0.018324
28      0.020291
29      0.021417
          ...   
8889    0.003185
8890    0.001377
8891    0.004656
8892    0.002008
8893    0.003919
8894    0.005468
8895    0.001670
8896    0.001761
8897    0.001791
8898    0.002971
8899    0.003059
8900    0.002763
8901    0.005232
8902    0.007369
8903    0.005951
8904    0.004680
8905    0.002784
8906    0.004683
8907    0.009147
8908    0.012419
8909    0.007354
8910    0.006334
8911    0.006192
8912    0.004900
8913    0.006418
8914    0.009763
8915    0.011514
8916    0.0141

In [112]:
ols = sm.OLS(df['diff'], df['df_normalized'])
results = ols.fit()
print results.summary()

                            OLS Regression Results                            
Dep. Variable:                   diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     11.19
Date:                Wed, 27 Apr 2016   Prob (F-statistic):           0.000828
Time:                        04:04:35   Log-Likelihood:                -21085.
No. Observations:                8919   AIC:                         4.217e+04
Df Residuals:                    8918   BIC:                         4.218e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
df_normalized    -1.3301      0.398     -3.344